In [1]:
from rosemary import jpt_setup; jpt_setup()
import os; os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [2]:

import logging
import math
from pathlib import Path
import os
import sys
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional
import json
import numpy as np
import pickle

import pyarrow
import datasets
import evaluate
import torch
from datasets import load_dataset, IterableDataset

import numpy as np

import transformers
from transformers import (
    CONFIG_MAPPING,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    DataCollatorForLanguageModeling,
    is_torch_tpu_available,
    set_seed,
)
from transformers.testing_utils import CaptureLogger
from transformers.trainer_utils import get_last_checkpoint
from transformers.trainer_callback import TrainerState
from transformers.trainer import TRAINER_STATE_NAME
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from transformers.models.gpt2.configuration_gpt2 import GPT2Config

from doremi.training_args import ModelArguments, DataTrainingArguments, FullTrainingArguments
import doremi.dataloader as data_utils
from doremi.trainer import DoReMiTrainer
try:
    import doremi.models as doremi_models
except Exception:
    
    pass
try:
    from flash_attn.models.gpt_neox import gpt_neox_config_to_gpt2_config
except Exception:
    pass


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.27.0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/language-modeling/requirements.txt")

logger = logging.getLogger(__name__)

[2023-08-01 14:56:14,094] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
package_dir = "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi"
cache_dir = '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache'
preprocessed_data = "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed"

envs = {
    "CACHE": cache_dir,
    "DOREMI_DIR": package_dir,
    "PILE_DIR": os.path.join(package_dir, "data", 'raw'),
    "PREPROCESSED_PILE_DIR": preprocessed_data,
    "MODEL_OUTPUT_DIR": os.path.join(package_dir, 'results'),
    "PARTITION": "el8",
    "HF_HOME": cache_dir,
    "TRANSFORMERS_CACHE": cache_dir,
    "HF_DATASETS_CACHE": cache_dir,
    "HF_DATASETS_IN_MEMORY_MAX_SIZE": "0",
    "TORCH_EXTENSIONS_DIR": cache_dir,
    "TMPDIR": cache_dir,
    "WANDB_DIR": os.path.join(cache_dir, "wandb"),
    "PREPROCESSED_DATA": preprocessed_data,
    'PREPROCESSED_CACHE': os.path.join(cache_dir, 'preprocessed_cache', 'perdomain_pile_preprocessed'),

}

for k, v in envs.items():
    os.environ[k] = v
    
os.makedirs(cache_dir, exist_ok=True)


In [4]:
print(';'.join([f'export {k}={v}' for k, v in envs.items()]))

export CACHE=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export DOREMI_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi;export PILE_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/raw;export PREPROCESSED_PILE_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed;export MODEL_OUTPUT_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/results;export PARTITION=el8;export HF_HOME=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export TRANSFORMERS_CACHE=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export HF_DATASETS_CACHE=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export HF_DATASETS_IN_MEMORY_MAX_SIZE=0;export TORCH_EXTENSIONS_DIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export TMPDIR=/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache;export WANDB_DIR=/gpfs/

In [6]:
# populate domain weight config
import json
domain_config_path = '../configs/humanmix_baseline_50kvocab.json'
domain_weights = {"cot": .25, "flan_v2": .25, "dolly": .25, "oasst1": .25}
domain_config = {"train_domain_weights": domain_weights, "eval_domain_weights": domain_weights}
with open(domain_config_path, 'w') as f:
    json.dump(domain_config, f)

In [11]:
# # move domain datasets to the right folder.
# src_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed'
# tgt_dir = preprocessed_data

# def copyanything(src, dst):
#     import shutil, errno
#     try:
#         shutil.copytree(src, dst)
#     except OSError as exc: # python >2.5
#         if exc.errno in (errno.ENOTDIR, errno.EINVAL):
#             shutil.copy(src, dst)
#         else: raise

# for domain in  domain_weights.keys():
#     src_domain_dir = os.path.join(src_dir, domain)
#     tgt_domain_dir = os.path.join(tgt_dir, domain)
#     print(src_domain_dir, os.path.isdir(src_domain_dir))


/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/cot True
/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/flan_v2 True
/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/dolly True
/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/oasst1 True


In [203]:
job_name = 'train_baseline'

nodes = 1
num_gpus = 1

model_name_or_path = 'gpt2'
cache_dir = envs['CACHE']
domain_config_path = '../configs/humanmix_baseline_50kvocab.json'
output_dir = os.path.join(envs['MODEL_OUTPUT_DIR'], job_name)
dataset_dir = envs['PREPROCESSED_CACHE']
dataset_dir = preprocessed_data
 
total_batch_size = 2 # # 64*8=512
per_device_train_batch_size = 2
gradient_accumulation_steps = 1
gradient_accumulation_steps = int(total_batch_size/(num_gpus*nodes)/per_device_train_batch_size)

max_steps = 200000; save_steps = 10000

# --learning_rate 1e-3 \
# --lr_end 1e-4 \
# --weight_decay 0.01 \
# --max_grad_norm 1.0 \
# --adam_epsilon 1e-8 \
# --lr_scheduler_name linear_warmup_cosine \
# --warmup_ratio 0.06 \

cmd = f"""
--dataset_name=pile \
--model_type={model_name_or_path} \
--tokenizer_name=gpt2 \
--do_train \
--cache_dir={cache_dir} \
--dataset_dir={dataset_dir} \
--domain_config_path={domain_config_path} \
--output_dir={output_dir} \
--max_token_length=1024 \
--per_device_train_batch_size={per_device_train_batch_size} \
--gradient_accumulation_steps={gradient_accumulation_steps} \
--dataloader_num_workers=1 \
--learning_rate=2e-5 \
--lr_scheduler_type=linear \
--warmup_ratio=0.03 \
--weight_decay=0. \
--max_grad_norm=1.0 \
--max_steps={max_steps} \
--evaluation_strategy=no \
--save_strategy=steps \
--save_steps={save_steps} \
--save_total_limit=1 \
--run_name={job_name} \
--seed=1111 \
--logging_strategy=steps \
--logging_steps=10 \
--logging_first_step \
--report_to=tensorboard \
--optim=adamw_hf \
--adam_beta1=0.9 \
--adam_beta2=0.99 \
--add_domain_id True \
--do_padding True \
--fp16 \
--config_overrides="n_positions=1024,n_embd=1024,n_layer=18,n_head=16" \
"""
# add_domain_id: for non-pile preprocessed dataset
# do_padding: true for variable size sequences, as in instruction tuning datasets.


import shlex
args = shlex.split(cmd)

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, FullTrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)
model_args, data_args, training_args

[INFO|training_args.py:1301] 2023-08-01 17:24:18,841 >> Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
[INFO|training_args.py:1716] 2023-08-01 17:24:18,842 >> PyTorch: setting up devices


(ModelArguments(model_name_or_path=None, model_type='gpt2', config_overrides='n_positions=1024,n_embd=1024,n_layer=18,n_head=16', config_name=None, tokenizer_name='gpt2', cache_dir='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache', use_fast_tokenizer=True, model_revision='main', use_auth_token=False, torch_dtype=None),
 DataTrainingArguments(dataset_dir='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed', dataset_name='pile', max_train_samples=None, max_eval_samples=None, max_token_length=1024, block_size=None, overwrite_cache=False, do_padding=True, add_domain_id=True, preprocessing_num_workers=None, shuffle=True),
 FullTrainingArguments(output_dir='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/results/train_baseline', overwrite_output_dir=False, do_train=True, do_eval=False, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_b

In [204]:

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

if training_args.should_log:
    # The default of training_args.log_level is passive, so we set log level at info here to have that default.
    transformers.utils.logging.set_verbosity_info()
    

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()


# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

08/01/2023 17:24:19 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
08/01/2023 17:24:19 - INFO - __main__ - Training/evaluation parameters FullTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.99,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=1,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
domain_config_path=../configs/humanmix_baseline_50kvocab.json,
doremi_optimizer=doremiv1,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsd

In [205]:

# Detecting last checkpoint.
last_checkpoint = None
num_skip_examples = 0
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )
        state = TrainerState.load_from_json(str(Path(last_checkpoint) / TRAINER_STATE_NAME))
        global_batch_size = training_args.train_batch_size * training_args.gradient_accumulation_steps * training_args.world_size
        num_skip_examples = state.global_step * global_batch_size
        logger.info(f"Skipping {num_skip_examples} examples")
        
last_checkpoint, num_skip_examples

ValueError: Output directory (/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/results/train_baseline) already exists and is not empty. Use --overwrite_output_dir to overcome.

In [ ]:

# Set seed before initializing model.
set_seed(training_args.seed)

In [ ]:

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

config_kwargs = {
    "cache_dir": model_args.cache_dir,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, **config_kwargs)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, **config_kwargs)
    if model_args.model_type == 'gpt_neox_flash':
        config = gpt_neox_config_to_gpt2_config(config)
        config.use_flash_attn = True
        config.fused_mlp = True
        config.fused_bias_fc = True
        config.fused_dropout_add_ln = True
        config.pad_vocab_size_multiple = 8
        config.activation_function = 'gelu_new'
        config.n_inner = None
        # disable absolute
        config.max_position_embeddings = 0
else:
    if model_args.model_type == 'gpt_flash': 
        config = GPT2Config(
                vocab_size=50257, n_positions=2048, n_embd=2048,
                n_layer=24, n_head=16, 
                scale_attn_by_inverse_layer_idx=True, 
                rotary_emb_fraction=0.5,
                use_flash_attn=True, fused_mlp=True,
                fused_bias_fc=True, fused_dropout_add_ln=True, 
                pad_vocab_size_multiple=8)
        # disable absolute
        config.max_position_embeddings = 0
    elif model_args.model_type == 'gpt_neox_flash':
        # convert to GPT2 config
        config = CONFIG_MAPPING['gpt_neox']() 
        config = gpt_neox_config_to_gpt2_config(config)
        config.use_flash_attn = True
        config.fused_mlp = True
        config.fused_bias_fc = True
        config.fused_dropout_add_ln = True
        config.pad_vocab_size_multiple = 8
        config.activation_function = 'gelu_new'
        config.n_inner = None
        # disable absolute
        config.max_position_embeddings = 0
    else:
        config = CONFIG_MAPPING[model_args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")
    if model_args.config_overrides is not None:
        logger.info(f"Overriding config: {model_args.config_overrides}")
        config.update_from_string(model_args.config_overrides)
        logger.info(f"New config: {config}")


In [ ]:

tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}

if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)

elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )
    
tokenizer

In [ ]:

if model_args.model_name_or_path:
    torch_dtype = (
        model_args.torch_dtype
        if model_args.torch_dtype in ["auto", None]
        else getattr(torch, model_args.torch_dtype)
    )
    if model_args.model_type in {'gpt_flash', 'gpt_neox_flash'}:
        model = doremi_models.GPTFlashAttnLMHeadModel.from_pretrained(model_args.model_name_or_path, config=config)
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_args.model_name_or_path,
            from_tf=bool(".ckpt" in model_args.model_name_or_path),
            config=config,
            cache_dir=model_args.cache_dir,
            revision=model_args.model_revision,
            use_auth_token=True if model_args.use_auth_token else None,
            torch_dtype=torch_dtype,
        )
else:
    if model_args.model_type in {'gpt_flash', 'gpt_neox_flash'}:
        model = doremi_models.GPTFlashAttnLMHeadModel(config)
    else:
        model = AutoModelForCausalLM.from_config(config)

    n_params = sum({p.data_ptr(): p.numel() for p in model.parameters()}.values())
    logger.info(f"Training new model from scratch - Total size={n_params/2**20:.2f}M params")
    
model

In [206]:

with open(training_args.domain_config_path, 'r') as f:
    domain_config = json.load(f)
    
domain_config['train_domain_weights'] =  {'cot': 0.5, 'flan_v2': 0.25, 'dolly': 0.12, 'oasst1': 0.13}


train_domain_weights_dict = domain_config['train_domain_weights']
eval_domain_weights_dict = domain_config['eval_domain_weights']
# whenever we convert dict to array, we sort by key
domain_list = list(sorted(train_domain_weights_dict.keys()))
num_domains = len(domain_list)


domain_list, num_domains, train_domain_weights_dict

(['cot', 'dolly', 'flan_v2', 'oasst1'],
 4,
 {'cot': 0.5, 'flan_v2': 0.25, 'dolly': 0.12, 'oasst1': 0.13})

In [207]:

if training_args.do_train:
    # data script could change tokenizer shape
    train_dataset = data_utils.get_preprocessed_mixed_dataset(
            preprocessed_dir=data_args.dataset_dir,
            domain_weights_dict=train_domain_weights_dict,
            dataset_name=data_args.dataset_name,
            cache_dir=model_args.cache_dir,
            split='train',
            max_samples=data_args.max_train_samples,
            add_domain_id=data_args.add_domain_id,
            tmp_file=None,
            seed=training_args.seed,
            tokenizer=tokenizer,
            shuffle=data_args.shuffle,
            num_skip_examples=num_skip_examples,
            shard_reversal=training_args.reweight_domains
    )

# if training_args.do_eval:
#     eval_dataset = data_utils.get_preprocessed_mixed_dataset(
#             preprocessed_dir=data_args.dataset_dir,
#             domain_weights_dict=eval_domain_weights_dict,
#             dataset_name=data_args.dataset_name,
#             cache_dir=model_args.cache_dir,
#             split='validation',
#             add_domain_id=data_args.add_domain_id,
#             max_samples=data_args.max_eval_samples,
#             tokenizer=tokenizer,
#             no_interleave=True)

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed/train'

In [215]:
from doremi.dataloader import determine_skip_per_domain


preprocessed_dir=data_args.dataset_dir
domain_weights_dict=train_domain_weights_dict
dataset_name=data_args.dataset_name
cache_dir=model_args.cache_dir
split='train'
max_samples=data_args.max_train_samples
add_domain_id=data_args.add_domain_id
tmp_file=None
seed=training_args.seed
tokenizer=tokenizer
shuffle=data_args.shuffle
num_skip_examples=num_skip_examples
shard_reversal=training_args.reweight_domains
no_interleave=False

print(preprocessed_dir)
print(domain_weights_dict)
print(dataset_name)
print(cache_dir)
print(split)
print(max_samples)
print(add_domain_id)
print(seed)
print(shuffle)
print(num_skip_examples)
print(shard_reversal)


domain_names = list(sorted(domain_weights_dict.keys()))
domain_to_idx = {domain_names[i]: i for i in range(len(domain_names))}
domain_weights = np.asarray([domain_weights_dict[domain_name] for domain_name in domain_names])
domain_weights = domain_weights / domain_weights.sum()

print()
print(json.dumps({'domain_names': domain_names, 
                  'domain_to_idx': domain_to_idx, 
                  'domain_weights': list(domain_weights)},
                indent=4))


# write domain weights to file if tmp_file is set
if tmp_file is not None:
    probabilities_tmp_file = tmp_file

    with open(str(probabilities_tmp_file), 'wb') as f:
        pickle.dump(domain_weights, f)
    probabilities = None
else:
    probabilities = domain_weights
    probabilities_tmp_file = None


print()
print(json.dumps({'probabilities': list(probabilities)}, indent=4))

# from doremi.dataloader import get_perdomain_datasets
# all_ds = get_perdomain_datasets(
#     preprocessed_dir, 
#     domain_weights_dict,
#     cache_dir=cache_dir,
#     split=split,
#     seed=seed,
#     domain_weights=domain_weights,
#     domain_names=domain_names,
#     num_skip_examples=num_skip_examples,
#     shuffle=shuffle,
#     shard_reversal=shard_reversal
# )

domain_name_to_skip_num = determine_skip_per_domain(num_skip_examples, seed, domain_weights, domain_names)

preprocessed_dir = Path(preprocessed_dir)
if split is not None and (preprocessed_dir / split).exists():
    preprocessed_dir = preprocessed_dir / split
else:
    logger.warn(f"No split used or split directory not found: using same data for all splits.")

domains = list(sorted(domain_weights_dict.keys()))

print(preprocessed_dir)
print(domain_name_to_skip_num)
print()
print(json.dumps({'preprocessed_dir': str(preprocessed_dir), 
                  'domain_name_to_skip_num': domain_name_to_skip_num}, indent=4))


all_ds = {}
for domain in domains:
    domain_dir = preprocessed_dir / domain
    if (domain_dir / f'{domain}_data.jsonl').exists():
        from datasets import load_dataset
        from functools import partial
        from open_instruct.finetune_trainer import encode_with_prompt_completion_format, encode_with_messages_format
        from doremi.dataloader import skippable_data_gen_dataset

        data_files = {'train': str(domain_dir / f'{domain}_data.jsonl')}
        raw_datasets = load_dataset(
            "json",
            data_files=data_files,
            cache_dir=model_args.cache_dir,
            use_auth_token=True if model_args.use_auth_token else None,
        )
        # Preprocessing the datasets.
        if "prompt" in raw_datasets["train"].column_names and "completion" in raw_datasets["train"].column_names:
            encode_function = partial(
                encode_with_prompt_completion_format,
                tokenizer=tokenizer,
                max_seq_length=1024,
            )
        elif "messages" in raw_datasets["train"].column_names:
            encode_function = partial(
                encode_with_messages_format,
                tokenizer=tokenizer,
                max_seq_length=1024,
            )
        else:
            raise ValueError("You need to have either 'prompt'&'completion' or 'messages' in your column names.")

        with training_args.main_process_first(local=False, desc="Processing instruction data"):
            lm_datasets = raw_datasets.map(
                encode_function,
                num_proc=16,
                batched=False,
            )
            lm_datasets.set_format(type="pt")
        ds = lm_datasets['train']
        ds = IterableDataset.from_generator(
                skippable_data_gen_dataset,
                gen_kwargs={'ds': ds,
                            'num_skip_examples': domain_name_to_skip_num[domain],
                            'loop': (split == 'train'),
                            'seed': seed,
                            'shuffle': shuffle}
                )
        seed += 1
    elif (domain_dir / 'dataset_info.json').exists():
        ds = load_from_disk(dataset_path=str(domain_dir))
        logger.info(f"Loaded {domain_dir}. Length: {len(ds)}")
    else:
        curr_shards = list(domain_dir.iterdir())
        if shard_reversal:
            curr_shards = list(reversed(curr_shards))
        # shuffle shard order
        random.Random(seed).shuffle(curr_shards)
        ds = IterableDataset.from_generator(
                skippable_data_gen,
                gen_kwargs={'shards': curr_shards,
                            'num_skip_examples': domain_name_to_skip_num[domain],
                            'loop': (split == 'train'),
                            'seed': seed,
                            'shuffle': shuffle}
                )
        seed += 1
    all_ds[domain] = ds

/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed
{'cot': 0.5, 'flan_v2': 0.25, 'dolly': 0.12, 'oasst1': 0.13}
pile
/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache
train
None
True
1111
True
0
False

{
    "domain_names": [
        "cot",
        "dolly",
        "flan_v2",
        "oasst1"
    ],
    "domain_to_idx": {
        "cot": 0,
        "dolly": 1,
        "flan_v2": 2,
        "oasst1": 3
    },
    "domain_weights": [
        0.5,
        0.12,
        0.25,
        0.13
    ]
}

{
    "probabilities": [
        0.5,
        0.12,
        0.25,
        0.13
    ]
}
08/01/2023 17:30:58 - WARNING - __main__ - No split used or split directory not found: using same data for all splits.
/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed
{'cot': 0, 'dolly': 0, 'flan_v2': 0, 'oasst1': 0}

{
    "preprocessed_dir": "/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/data/processed",
    "domai

/tmp/ipykernel_64564/969478157.py:79: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(f"No split used or split directory not found: using same data for all splits.")


  0%|          | 0/1 [00:00<?, ?it/s]

08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #0 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-2b9a63c60434c0f7_00000_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #1 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-2b9a63c60434c0f7_00001_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #2 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-2b9a63c60434c0f7_00002_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #3 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mi

  0%|          | 0/1 [00:00<?, ?it/s]

08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #0 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-273c81b3f5a0e1d5_00000_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #1 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-273c81b3f5a0e1d5_00001_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #2 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-68fd04897f8e942c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-273c81b3f5a0e1d5_00002_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #3 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mi

  0%|          | 0/1 [00:00<?, ?it/s]

08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #0 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-216e0143932edb57_00000_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #1 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-216e0143932edb57_00001_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #2 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-216e0143932edb57_00002_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #3 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mi

  0%|          | 0/1 [00:00<?, ?it/s]

08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #0 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-d828a53418d0df31_00000_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #1 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-d828a53418d0df31_00001_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #2 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-d43591dedd2996a1/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-d828a53418d0df31_00002_of_00016.arrow
08/01/2023 17:30:58 - INFO - datasets.arrow_dataset - Process #3 will write at /gpfs/u/scratch/PTFM/PTFMqngp/github/mi

In [216]:
# test_ds = load_dataset(
#     "json",
#     data_files='test.jsonl',
#     cache_dir=model_args.cache_dir)['train']
# test_ds[0]

# for x in test_ds.to_iterable_dataset():
#     print(x)
# for i, v in enumerate(ds):
#     if i == 10:
#         break
#     print(v)

In [217]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def add_domain_id_generator(ds, domain_idx):
    for ex in ds:
        print(ex)
        ex['domain_id'] = domain_idx
        yield ex

        
domain_ds_ls = []
for domain_name in domain_names:
    domain_idx = domain_to_idx[domain_name]
    domain_ds = all_ds[domain_name]
    # add domain_id if necessary
    if add_domain_id:
        domain_ds = IterableDataset.from_generator(
            add_domain_id_generator, 
            gen_kwargs={'ds': domain_ds, 'domain_idx': domain_idx})
    domain_ds_ls.append(domain_ds)


08/01/2023 17:30:59 - INFO - datasets.builder - Using custom data configuration default-054ef3928e56a804
08/01/2023 17:30:59 - INFO - datasets.info - Loading Dataset Infos from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/generator
08/01/2023 17:30:59 - INFO - datasets.builder - Using custom data configuration default-ad2f540c72bd8fb9
08/01/2023 17:30:59 - INFO - datasets.info - Loading Dataset Infos from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/generator
08/01/2023 17:30:59 - INFO - datasets.builder - Using custom data configuration default-dfe585457d1ad863
08/01/2023 17:30:59 - INFO - datasets.info - Loading Dataset Infos from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/generator
08/01/2023 17:30:59 - INFO - datasets.builder - Using custom data configuration default-253e20

In [218]:
from doremi.dataloader import interleave_datasets
no_interleave = False

if no_interleave:
    # instead of interleaving, run through each dataset
    def data_generator(shards):
        for shard in shards:
            for ex in shard:
                yield ex
    ds = IterableDataset.from_generator(data_generator, gen_kwargs={'shards': domain_ds_ls})
    logger.info("Not interleaving dataset - will not sample according to domain weights")

else:
    ds = interleave_datasets(
            domain_ds_ls,
            probabilities=probabilities,
            probabilities_file=probabilities_tmp_file,
            seed=seed)
    

def take_data_generator(ds, max_samples):
    idx = 0
    for ex in ds:
        yield ex
        idx += 1
        if max_samples is not None and idx >= max_samples:
            return

ds = IterableDataset.from_generator(take_data_generator, gen_kwargs={'ds': ds, 'max_samples': max_samples})
train_dataset = ds

08/01/2023 17:30:59 - INFO - datasets.builder - Using custom data configuration default-d63725ddb00f354b
08/01/2023 17:30:59 - INFO - datasets.info - Loading Dataset Infos from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/generator


In [219]:
# ds_iter = iter(ds)

# l = []

# for i, x in enumerate(ds_iter):
# #     print(x.keys())
#     l.append(x['domain_id'])
#     if i == 2000:
#         break
        
# from collections import Counter
# Counter(l)

In [220]:

# turn off find unused parameters
training_args.ddp_find_unused_parameters = False

# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
# embedding_size = model.get_input_embeddings.weight.shape[0]
# if len(tokenizer) > embedding_size:
#     model.resize_token_embeddings(len(tokenizer))

torch.cuda.empty_cache()

# Initialize our Trainer
trainer = DoReMiTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_utils.get_data_collator(tokenizer, do_padding=data_args.do_padding),
)

[INFO|trainer.py:565] 2023-08-01 17:31:00,383 >> max_steps is given, it will override any value given in num_train_epochs


In [221]:

# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    trainer.save_model()  # Saves the tokenizer too for easy upload

    metrics = train_result.metrics

    if training_args.reweight_domains:
        avg_domain_weights_dict = {}
        for i in range(len(model.avg_domain_weights)):
            domain_name = domain_list[i]
            metrics[f'avg_domain_weight:{domain_name}'] = model.avg_domain_weights[i].item()
            avg_domain_weights_dict[domain_name] = model.avg_domain_weights[i].item()

        # save avg domain weights to json
        avg_domain_weights_file = Path(training_args.output_dir) / 'avg_domain_weights.json'
        with open(avg_domain_weights_file, 'w') as f:
            json.dump(avg_domain_weights_dict, f, indent=2)

        # also save to configs dir
        config_dict = {"train_domain_weights": avg_domain_weights_dict,
                       "eval_domain_weights": avg_domain_weights_dict}
        config_dict_file = Path(__file__).parent.parent / 'configs' / f"{Path(training_args.output_dir).name}.json"
        with open(config_dict_file, 'w') as f:
            json.dump(config_dict, f, indent=2)

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

[INFO|trainer.py:1682] 2023-08-01 17:31:01,721 >> ***** Running training *****
[INFO|trainer.py:1683] 2023-08-01 17:31:01,722 >>   Num examples = 400,000
[INFO|trainer.py:1684] 2023-08-01 17:31:01,723 >>   Num Epochs = 9,223,372,036,854,775,807
[INFO|trainer.py:1685] 2023-08-01 17:31:01,723 >>   Instantaneous batch size per device = 2
[INFO|trainer.py:1688] 2023-08-01 17:31:01,724 >>   Total train batch size (w. parallel, distributed & accumulation) = 2
[INFO|trainer.py:1689] 2023-08-01 17:31:01,724 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:1690] 2023-08-01 17:31:01,725 >>   Total optimization steps = 200,000
[INFO|trainer.py:1691] 2023-08-01 17:31:01,727 >>   Number of trainable parameters = 279,245,824


08/01/2023 17:31:01 - INFO - datasets.arrow_dataset - Caching indices mapping at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-1d505b89c64f68c7.arrow
{'dataset': 'cot', 'id': 'cot_60290', 'messages': [{'role': 'user', 'content': 'QUESTION: Premise: "A man is squatting down while painting his artwork on a large metal garage door."\nBased on this premise, can we conclude that the hypothesis "A man paints art on garage door." is true?\nOptions:\n- yes\n- it is not possible to tell\n- no\n\nLet\'s solve it slowly: A man is squatting down while paints art on metal garage door.\nThe answer is yes.\n[QUESTION] Premise: "Woman hiking outside in a vineyard."\nBased on this premise, can we conclude that the hypothesis "The vines are covered with ripe grapes." is true?\nOptions:\n- yes\n- it is not possible to tell\n- no\nHiking in a vineyard does not imply the vines 

        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'dataset': 'cot', 'id': 'cot_69782', 'messages': [{'role': 'user', 'content': 'People dancing outside does not mean that those people had attended a wedding.. So what could be the question?\nQuestion followed by answer: If "A group of men and women dancing outside." does that mean that "People dancing outside after a wedding."?\nOptions:\n- yes\n- it is not possible to tell\n- no\nit is not possible to tell\n\nOne can not hold an infant and eat food at the same time.\nThe question and answer are below.\nPremise: "A man holds an infant while leaning against a building."\nHypothesis: "A man is eating food."\nDo we know that the hypothesis entailed by the premise?\nno\n\nThe monster truck leaping over crushed cars is just a past tense action of how the truck leaps over crushed cars.\nThe question and answer are below.\nCan we conclude from "A giant monster truck leaps over crushed cars and into the air inside the arena." that "Mon

        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
08/01/2023 17:31:01 - INFO - datasets.arrow_dataset - Caching indices mapping at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-a01381664fd2589b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-3ef242118c55c9ad.arrow
{'dataset': 'flan_v2', 'id': 'flan_v2_70780', 'messages': [{'role': 'user', 'content': "(Question)\nInside Recordings is a Los Angeles California based independent record label founded by singer-songwriter Jackson Browne in 1999. Browne has stated that the mission of the label is to create a haven for music that might not find a home in the mainstream. In 2005 Inside Recordings signed a nationwide direct distribution deal with independent music distributor Alternative Distribution Alliance. The label's inaugural release under the agreement was Browne's own Solo Acoustic Vol. Given a list of categories: company, educational institution, artist, athlete, office ho

        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


...

        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'dataset': 'cot', 'id': 'cot_34935', 'messages': [{'role': 'user', 'content': 'Come up with a question and stream of consciousness reasoning that would justify this answer: it is not possible to tell\n'}, {'role': 'assistant', 'content': 'The question is: Test for natural language inference.\nPremise: "A guy playing a violin on the street and four women are listening to him."\nHypothesis: "Women tip a street performer."\nIs the hypothesis entailed by the premise?\nOptions:\n- yes\n- no\n- it is not possible to tell\nStream of consciousness: Guy playing a violin on the street is not necessarily a street performer.\n'}], 'input_ids': tensor([   27,    91,  7220,    91,    29,   198, 16773,   510,   351,   257,
         1808,   290,  4269,   286, 10510, 14607,   326,   561, 12051,   428,
         3280,    25,   340,   318,   407,  1744,   284,  1560,   198,    27,
           91,   562, 10167,    91,    29,   198,

        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'dataset': 'flan_v2', 'id': 'flan_v2_25674', 'messages': [{'role': 'user', 'content': 'Tullar et Berchielli (1982) ont également constaté que les jeunes se déplaçaient sur de plus grandes distances que les adultes.\n\nCould you please translate this to English?\n'}, {'role': 'assistant', 'content': 'Tullar and Berchielli (1982) also found that juveniles moved greater distances than adults.'}], 'input_ids': tensor([   27,    91,  7220,    91,    29,   198,    51,   724,   283,  2123,
         4312,  3043, 15516,   357, 30763,     8, 39585, 38251, 13528,   972,
         1500,   265,  2634,  8358, 10287, 11223,  4015,   384, 39073,   489,
           64, 50041,  1153,   969,   390,  5556,  4490,   274, 18868,  8358,
        10287,  4044,   274,    13,   198,   198, 23722,   345,  3387, 15772,
          428,   284,  3594,    30,   198,    27,    91,   562, 10167,    91,
           29,   198,    51,   724,   283,   290,  4312,  3043, 15516,   35

        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'dataset': 'oasst1', 'id': 'oasst1_14536', 'messages': [{'role': 'user', 'content': 'Necesito saber como medir las pulgadas de la rueda de mi bicicleta pero no tengo ni idea, ¿me puedes indicar cómo hacerlo? Tengo un metro en cm :('}, {'role': 'assistant', 'content': 'Asumo que lo que quieres obtener es el radio de la rueda de tu bicicleta, ¿es así?\n\nEn caso afirmativo, únicamente tendrías que tomar tu metro y medir desde el centro de la rueda (lo que se llama buje) hasta el extremo de esta. Dependiendo de para qué lo quieras, puedes tomar la medida hasta el aro de la llanta o hasta el neumático. Una vez tengas esa medida en centímetros, sólo tienes que dividir el valor que te haya salido entre 2.54 y el resultado será el radio de la rueda en pulgadas. \n\nSi, por el contrario, lo que querías era obtener la longitud de la circunferencia de la rueda, sólo tienes que añadir un paso extra al proceso que acabo de d

        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


AssertionError: Attempted unscale_ but _scale is None.  This may indicate your script did not use scaler.scale(loss or outputs) earlier in the iteration.

In [202]:
next(iter(train_dataset))

08/01/2023 17:23:41 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-c6ba3d14d25cb207.arrow
{'dataset': 'cot', 'id': 'cot_60290', 'messages': [{'role': 'user', 'content': 'QUESTION: Premise: "A man is squatting down while painting his artwork on a large metal garage door."\nBased on this premise, can we conclude that the hypothesis "A man paints art on garage door." is true?\nOptions:\n- yes\n- it is not possible to tell\n- no\n\nLet\'s solve it slowly: A man is squatting down while paints art on metal garage door.\nThe answer is yes.\n[QUESTION] Premise: "Woman hiking outside in a vineyard."\nBased on this premise, can we conclude that the hypothesis "The vines are covered with ripe grapes." is true?\nOptions:\n- yes\n- it is not possible to tell\n- no\nHiking in a vineyard do

{'dataset': 'cot',
 'id': 'cot_60290',
 'messages': [{'role': 'user',
   'content': 'QUESTION: Premise: "A man is squatting down while painting his artwork on a large metal garage door."\nBased on this premise, can we conclude that the hypothesis "A man paints art on garage door." is true?\nOptions:\n- yes\n- it is not possible to tell\n- no\n\nLet\'s solve it slowly: A man is squatting down while paints art on metal garage door.\nThe answer is yes.\n[QUESTION] Premise: "Woman hiking outside in a vineyard."\nBased on this premise, can we conclude that the hypothesis "The vines are covered with ripe grapes." is true?\nOptions:\n- yes\n- it is not possible to tell\n- no\nHiking in a vineyard does not imply the vines are covered with ripe grapes.\nThe answer is it is not possible to tell.\nQ: Test for natural language inference.\nPremise: "A woman walks through the drizzling rain while shielding herself with her red umbrella."\nHypothesis: "A woman is out in the rain."\nIs the hypothesis 

In [180]:
batch = list(train_dataset.take(3))
batch = [{'input_ids': x['input_ids'], 
          'attention_mask': x['attention_mask'], 
          'labels': x['labels'],
          'domain_id': x['domain_id']} for x in batch]

08/01/2023 17:11:00 - WARNING - datasets.arrow_dataset - Loading cached shuffled indices for dataset at /gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/doremi/cache/json/default-20e8d8f5df875937/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-c6ba3d14d25cb207.arrow
{'dataset': 'cot', 'id': 'cot_60290', 'messages': [{'role': 'user', 'content': 'QUESTION: Premise: "A man is squatting down while painting his artwork on a large metal garage door."\nBased on this premise, can we conclude that the hypothesis "A man paints art on garage door." is true?\nOptions:\n- yes\n- it is not possible to tell\n- no\n\nLet\'s solve it slowly: A man is squatting down while paints art on metal garage door.\nThe answer is yes.\n[QUESTION] Premise: "Woman hiking outside in a vineyard."\nBased on this premise, can we conclude that the hypothesis "The vines are covered with ripe grapes." is true?\nOptions:\n- yes\n- it is not possible to tell\n- no\nHiking in a vineyard do

{'dataset': 'flan_v2', 'id': 'flan_v2_70780', 'messages': [{'role': 'user', 'content': "(Question)\nInside Recordings is a Los Angeles California based independent record label founded by singer-songwriter Jackson Browne in 1999. Browne has stated that the mission of the label is to create a haven for music that might not find a home in the mainstream. In 2005 Inside Recordings signed a nationwide direct distribution deal with independent music distributor Alternative Distribution Alliance. The label's inaugural release under the agreement was Browne's own Solo Acoustic Vol. Given a list of categories: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work, what category does the paragraph belong to?\n(Answer)\nCompany\n\n\n(Question)\nI’d Rather Go Blonde is the eighth album in a twenty-year career that has seen Eleanor McEvoy establish herself as one of Ireland’s most accom

In [192]:
data_args.do_padding = True
collator = data_utils.get_data_collator(tokenizer, do_padding=data_args.do_padding)
collator


<function doremi.dataloader.get_data_collator.<locals>.data_collator(features)>

In [194]:
x = collator(batch)
x['input_ids'].shape, x['attention_mask'].shape

(torch.Size([3, 568]), torch.Size([3, 568]))

In [185]:
for x in batch:
    for k,v in x.items():
        if k.startswith('domain'):
            print(k, v)
        else:
            print(k, v.shape)

input_ids torch.Size([568])
attention_mask torch.Size([568])
labels torch.Size([568])
domain_id 0
input_ids torch.Size([378])
attention_mask torch.Size([378])
labels torch.Size([378])
domain_id 2
input_ids torch.Size([493])
attention_mask torch.Size([493])
labels torch.Size([493])
domain_id 0
